# Parent Hamiltonian Benchmark

Present notebook documents the **Parent Hamiltonian** (**PH**) **kernel** and its associated **Benchmark Test Case**. Additionally, explains how the benchmark can be executed.

$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

## 1. PH Kernel

Given an ansatz with a state $\ket{\Psi \left(\theta\right)}$, where $\theta$ is a vector of parameters, the **PH kernel** aims to compute a Hamiltonian, $\mathcal{H}$, such that the ansatz is its ground state with a fixed energy of 0. Such Hamiltonian is called the **Parent Hamiltonian**^of the ansatz (or of the state $\ket{\Psi \left(\theta\right)}$)

$$\mathcal{H}\ket{\Psi \left(\theta\right)} = 0 \tag{1}$$

In the **TNBS** it is expected to use the **PH kernel** for evaluating the performance of quantum architectures for running typical **VQE** circuits (or ansatzes).

This kernel is based on the original *Parent Hamiltonian*  paper:

* Kobayashi, F., Mitarai, K., & Fujii, K. (2022). Parent Hamiltonian as a benchmark problem for variational quantum eigensolvers. Phys. Rev. A, 105, 052415 (https://doi.org/10.1103%2Fphysreva.105.052415)

## 2. BTC

As a **BTC** for the **PH kernel**  we are going to use the proposed ansatz on the original *Parent Hamiltonian*  paper that is depicted in the following figure:


![alternatvie text](./parent_hamiltonian.svg)


The ansatz is the circuit inside the rectangle solid line and it can be composed of several layers. Each layer (rectangle dashed line) is composed of a $R_X(\theta_i)$ gate by qubit, followed by a ladder of $CNOT$ gates and finally a $R_Z(\theta_{i+1})$ by qubit.

The angles of the different gates will be set to, follow the TNBS guidelines:

$$\delta \theta = \frac{\pi}{4*(n_l+1)}$$ $$\theta_i = (i+1) \delta \theta \; i=0, 1, \cdots 2n_l-1$$ where $n_l$ is the number of layers of the ansatz (the **depth** of the ansatz)

For a fixed **number of qubits** and **depth** the corresponding **PH** of the ansatz should be computed and decomposed as linear decomposition following $(2)$: $$\mathcal{H} = \sum_{I=0}^{4^n-1} a_I P_{I}^n\tag{2}$$ where $P_I^n$ is a generalized Pauli string: $$P_I^n=\sigma_{i_0} \otimes \sigma_{i_1} \cdots \otimes \sigma_{i_{n-1}}$$ where $i_j={0, 1, 2, 3}$.

For each possible **Pauli** string the complete ansatz circuit should be executed and the corresponding expected value of the **Pauli** string should be computed (this is depicted in the figure with *Rotation to $P_i$ measurement basis* and the *$\langle P_i \rangle$* boxes). Finally, all the computed expected values ($\langle P_i \rangle$) should be gathered and the ground state energy ($E_0$) of the ansatz under the **PH** should be computed using $(3)$:$$E_0 = \sum_{I=0}^{4^n-1} a_I \langle P_i \rangle \tag{3}$$

If all properly works the obtained $E_0$ should be as close to 0 as possible.

## 3 Benchmark execution.

In the present repository, we have computed the Pauli decomposition for ansatz circuits from 3 to 30 qubits and for 1 to 4 depths. The mandatory files are stored inside the **BTC_04_PH/configuration_files/**. There are 2 kinds of files:

* *_parameters.csv* files: where the different parameters of the ansatz circuit are stored.
* _pauli.csv* files: where the corresponding Pauli decomposition is stored.

These files are mandatory for executing the **BTC** using the code of the repository.

## 4. my_benchmark_execution.py

The **my_benchmark_execution.py** module executes the benchmark for the **PH BTC**. This module needs the files stored in the **BTC_04_PH/configuration_files/** folder. 

The benchmark can be configured using the *kernel_configuration* and the *benchmark_arguments* dictionary at the end of the **my_benchmark_execution.py** module. 

For selecting the depths to execute the key *depth* from the *kernel_configuration* dictionary should be modified (a list with the different depths should be provided). Only depths from 1 to 4 can be provided (are the precomputed ansatz depths).
For selecting the number of qubits to execute the key *list_of_qbits* from the *benchmark_arguments* should be modified (a list with different number of qubits should be provided). Only qubits from 3 to 30 can be provided (are the precomputed ansatz qubits).

**NOTE** 
The bechmark will be executed for each number of qubits and depths. So if you provide:
* depth : [2, 3]
*  list_of_qbits: [15, 18]

then the ansatzes for 15 qubits and 2 and 3 depths, and for 18 and 2 of 3 ansatzes will be executed (4 different executions will be done).


## 5. Generating the JSON file.

Once the files from a complete benchmark execution are generated the information should be formated following the **NEASQC JSON schema**. For doing this the **neasqc_benchmark.py** module can be used. At the end of the file the path to the folder where all the files from benchmark are stored should be provided to the variable **folder**.

For creating the JSON file following command should eb executed:

    python neasqc_benchmark.py

## 6. Complete Workflow.

The bash script **benchmark_exe.sh** allows to automatize the execution of the benchmark and the JSON file generation (once the *my_benchmark_execution.py* and the *neasqc_benchmark.py* are properly configured).

    bash benchmark_exe.sh

## 7. About the other notebooks.

The other notebooks presented in the **BTC_04_PH/PH/notebooks/* folder gives more information about the **Parent Hamiltonian** and about the code used for generating the configuration files. The user can check it and can use for creating different ansatzes and its corresponding Pauli decomposition even can be used for creating **BTC** with higher depths (or even higher numbers of qubits). The notebooks are:

* **02_Ansatzes.ipynb**: in this notebook the code for computing different ansatzes and their corresponding states are explained.
* **03_Using_PH_Class.ipynb**: in this notebook the internal details of the **Parent Hamiltonian** computation are provided. Additionally, the mandatory code for creating the Pauli decomposition of the **PH** is described.
* **04_ParentHamiltonian_execution.ipynb**: in this notebook the code used for, giving the ansatz and its corresponding **PH** Pauli decomposition, execute **VQE** step is explained.
* **05_CompleteWorkflow.ipynb**: in this notebook the code for a complete workflow (creating the ansatz, computing the PH and executing the **VQE** step) is provided and explained.
* **06_ParentHamiltonian_with_MPS.ipynb**: the main problem of the **PH** case is the need to compute the state of the ansatz for obtaining the **PH**. Using vector state simulators for this task can be very inefficient (and even unaffordable) when the number of qubits (or even the depth) increases. For this case techniques like *Matrix Product State* from **Tensor Networks** can be used. In this notebook, we explain how to use or naive implementation of these computations for obtaining the **PH** Pauli decomposition.
* **07_Ansatz_MPS.ipynb**: this notebook explains how to use our **MPS** implementation for computing the state of the **BTC** ansatz.
* **08_ReducedDensityMatriceswithMPS.ipynb**: this notebook shows how to use our **MPS** implementation for computing the **PH** Pauli decomposition.